In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline
import torch.nn as nn
import torch.nn.functional as F


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

True
1
0
GeForce RTX 2080 with Max-Q Design


In [3]:
# data_dir = '../../../../BottleStoodUp_atNight/Positive/'        #For the work laptop
data_dir = '../../../Images/BottleStoodUp_atNight/Positive'        #For the home laptop

In [4]:
# transform_characteristics = transforms.Compose([transforms.Resize(255),
#                                 transforms.CenterCrop(224),
#                                 transforms.ToTensor()])

transform_characteristics = transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize(255),
                                                transforms.CenterCrop(224)])
dataset = datasets.ImageFolder(data_dir, transform=transform_characteristics)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

We use only the encoder part of the nertwork

In [5]:
class Autoencoder_latentSpace(nn.Module):
    def __init__(self):
        super().__init__()        
        # N, 1, 28, 28
        # 32, 3, 224, 224
        input_channels = 3              # number of channels of the input image
        output_channels = 110           # ~= 224/2. Shape of the input image 
        kernel_size = 9
        padding_val = 1
        stride_val = 5

        output_channels_layer2 = output_channels*2+5

        output_channels_layer3 = output_channels_layer2*2
        
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size, stride=stride_val, padding=padding_val),         # input image channels, output channels, kernel size (filter). Dimension rseult: -> N, 110, 44, 44
            nn.ReLU(),
            nn.Conv2d(output_channels, output_channels_layer2, kernel_size, stride=stride_val, padding=padding_val), # -> N, 225, 8, 8
            nn.ReLU(),
            nn.Conv2d(output_channels_layer2, output_channels_layer3, 8) # -> N, 450, 1, 1
        )
        
        # self.show_modules()
        # self.show_weights()

    def forward(self, x):
        print("This is the forward function")
        encoded = self.encoder(x)
        return encoded
    
    def show_modules(self):
        print("This is the new function")
        # print(self.modules())
        i = 0
        for m in self.modules():
            print(m)
            print("i is: ", i)
            print("print the next module")
            i = i +1
    
    def show_weights(self):
        print("This is the function for showing the initial weights")
        # print(self.modules())
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if i == 0:
                    print("i is: ", i)
                    print("The type of the module weight is: ", type(m.weight))
                    print("The shape of the module weight is: ", m.weight.shape)
                    print("print the next module")
                    i = i +1
    def show_one_module_weights(self):
        print("Showing only one module's weights")
        # print(self.modules())
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if i == 0:
                    print("i is: ", i)
                    print("The weights are: ", m.weight)
                    i = i +1
 

In [6]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device

device = get_device()
print(device)

cuda:0


In [7]:
model_encoder = Autoencoder_latentSpace()
model_encoder.to(device)

Autoencoder_latentSpace(
  (encoder): Sequential(
    (0): Conv2d(3, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(110, 225, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(225, 450, kernel_size=(8, 8), stride=(1, 1))
  )
)

In [8]:
model_encoder.show_one_module_weights()

Showing only one module's weights
i is:  0
The weights are:  Parameter containing:
tensor([[[[ 4.4871e-02,  2.9082e-02,  3.2841e-02,  ..., -4.0282e-02,
           -1.7379e-02, -5.0551e-02],
          [-1.4576e-02, -2.8838e-02, -2.4488e-02,  ..., -6.0105e-02,
           -4.9931e-02, -6.2321e-02],
          [-5.7696e-02,  5.3239e-02,  5.0371e-02,  ...,  2.5325e-02,
           -3.2477e-02,  3.6619e-02],
          ...,
          [ 5.8053e-02,  1.9525e-02, -1.9289e-02,  ...,  3.8182e-02,
            2.5268e-02, -6.1149e-02],
          [ 5.3414e-02,  3.6838e-02, -6.2954e-02,  ...,  2.3757e-02,
           -4.4671e-02, -4.1561e-02],
          [ 3.3526e-02, -6.4007e-02, -8.9518e-03,  ...,  9.9564e-03,
           -1.3091e-02,  6.2309e-02]],

         [[-3.4604e-02,  2.8722e-02,  3.0524e-02,  ..., -3.1441e-02,
            2.1474e-04, -4.5121e-02],
          [ 6.2992e-02,  2.9784e-02, -3.2846e-02,  ...,  3.3754e-02,
           -5.3615e-02, -3.2559e-02],
          [-5.0462e-02, -3.7823e-02, -6.0304

In [9]:
class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()        
        # N, 1, 28, 28
        # 32, 3, 224, 224
        input_channels = 3              # number of channels of the input image
        output_channels = 110           # ~= 224/2. Shape of the input image 
        kernel_size = 9
        padding_val = 1
        stride_val = 5

        
        output_channels_layer2 = output_channels*2+5

        output_channels_layer3 = output_channels_layer2*2
        
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size, stride=stride_val, padding=padding_val),         # input image channels, output channels, kernel size (filter). Dimension rseult: -> N, 110, 44, 44
            nn.ReLU(),
            nn.Conv2d(output_channels, output_channels_layer2, kernel_size, stride=stride_val, padding=padding_val), # -> N, 225, 8, 8
            nn.ReLU(),
            nn.Conv2d(output_channels_layer2, output_channels_layer3, 8) # -> N, 450, 1, 1
        )
        
        # N , 450, 1, 1
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(output_channels_layer3, output_channels_layer2, 8),  # -> 32, 225, 8, 8
            nn.ReLU(),
            nn.ConvTranspose2d(output_channels_layer2, output_channels, kernel_size, stride=stride_val, padding=padding_val, output_padding=2), 
            nn.ReLU(),
            nn.ConvTranspose2d(output_channels, input_channels, kernel_size, stride=stride_val, padding=padding_val, output_padding=2), # N, 3, 224, 224
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded
    
 
# Note: nn.MaxPool2d -> use nn.MaxUnpool2d, or use different kernelsize, stride etc to compensate...
# Input [-1, +1] -> use nn.Tanh

In [10]:
model = Autoencoder()

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-3, 
                             weight_decay=1e-5)

In [11]:
model.to(device)

Autoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(110, 225, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(225, 450, kernel_size=(8, 8), stride=(1, 1))
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(450, 225, kernel_size=(8, 8), stride=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(225, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (3): ReLU()
    (4): ConvTranspose2d(110, 3, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (5): Sigmoid()
  )
)

In [12]:
filepath = "../../../BottlesAnomalies_TFM/models/pytorchModels/PytorchModel_withCUDA"
# For loading the model 
model.load_state_dict(torch.load(filepath))
model.eval()

Autoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(110, 225, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(225, 450, kernel_size=(8, 8), stride=(1, 1))
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(450, 225, kernel_size=(8, 8), stride=(1, 1))
    (1): ReLU()
    (2): ConvTranspose2d(225, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (3): ReLU()
    (4): ConvTranspose2d(110, 3, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1), output_padding=(2, 2))
    (5): Sigmoid()
  )
)

In [13]:
index = 0
i = 0
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        if i == index:
            print("i is: ", i)
            print("The weights are: ", m.weight)
        i = i +1

i is:  0
The weights are:  Parameter containing:
tensor([[[[-1.6085e-25, -1.6363e-25, -1.6663e-25,  ..., -1.7430e-25,
           -1.7557e-25, -1.7903e-25],
          [-2.2771e-25, -2.5256e-25, -2.5323e-25,  ..., -2.6019e-25,
           -2.5946e-25, -2.5337e-25],
          [-2.5053e-25, -2.6664e-25, -2.5478e-25,  ..., -2.7490e-25,
           -2.6799e-25, -2.6038e-25],
          ...,
          [-2.6943e-25, -2.8397e-25, -2.8483e-25,  ..., -3.0378e-25,
           -2.9532e-25, -2.8298e-25],
          [-2.8312e-25, -2.8671e-25, -2.8712e-25,  ..., -2.9985e-25,
           -2.9239e-25, -2.9911e-25],
          [-2.9684e-25, -2.9257e-25, -2.9676e-25,  ..., -3.0312e-25,
           -3.0702e-25, -3.0488e-25]],

         [[-1.6082e-25, -1.6399e-25, -1.6598e-25,  ..., -1.7378e-25,
           -1.7550e-25, -1.7926e-25],
          [-2.2830e-25, -2.5180e-25, -2.5322e-25,  ..., -2.6058e-25,
           -2.5928e-25, -2.5301e-25],
          [-2.5050e-25, -2.6661e-25, -2.5564e-25,  ..., -2.7503e-25,
         

In [14]:
layers_weights_list = []
for m in model.modules():
    if isinstance(m, nn.Conv2d):
        weights = m.weight
        layers_weights_list.append(weights)
print("The number of layers that the loaded model has, is: ", len(layers_weights_list))

The number of layers that the loaded model has, is:  3


Copying the module's weight from one model to another. 

In [15]:
class Autoencoder_latentSpace(nn.Module):
    def __init__(self):
        super().__init__()        
        # N, 1, 28, 28
        # 32, 3, 224, 224
        input_channels = 3              # number of channels of the input image
        output_channels = 110           # ~= 224/2. Shape of the input image 
        kernel_size = 9
        padding_val = 1
        stride_val = 5

        
        output_channels_layer2 = output_channels*2+5

        output_channels_layer3 = output_channels_layer2*2
        
        self.encoder = nn.Sequential(
            nn.Conv2d(input_channels, output_channels, kernel_size, stride=stride_val, padding=padding_val),         # input image channels, output channels, kernel size (filter). Dimension rseult: -> N, 110, 44, 44
            nn.ReLU(),
            nn.Conv2d(output_channels, output_channels_layer2, kernel_size, stride=stride_val, padding=padding_val), # -> N, 225, 8, 8
            nn.ReLU(),
            nn.Conv2d(output_channels_layer2, output_channels_layer3, 8) # -> N, 450, 1, 1
        )

    def forward(self, x):
        # print("This is the forward function")
        encoded = self.encoder(x)
        return encoded
    
    def show_modules(self):
        print("This is the show modules function")
        i = 0
        for m in self.modules():
            print(m)
            print("i is: ", i)
            print("print the next module")
            i = i +1
            
    def show_one_layer_weights(self, index):
        print("This is the one layer show function")
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                if i == index:
                    print("i is: ", i)
                    print("The weights are: ", m.weight)
                i = i +1
    
    def update_weights(self):
        print("updating weights function")
        i = 0
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight = layers_weights_list[i]
                i = i +1

                                        
 

In [16]:
model_encoder = Autoencoder_latentSpace()
model_encoder.to(device)
# model.eval()

Autoencoder_latentSpace(
  (encoder): Sequential(
    (0): Conv2d(3, 110, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(110, 225, kernel_size=(9, 9), stride=(5, 5), padding=(1, 1))
    (3): ReLU()
    (4): Conv2d(225, 450, kernel_size=(8, 8), stride=(1, 1))
  )
)

In [17]:
model_encoder.show_one_layer_weights(0)

This is the one layer show function
i is:  0
The weights are:  Parameter containing:
tensor([[[[-5.1781e-02, -4.8670e-02, -2.4642e-02,  ..., -1.2781e-02,
           -2.0488e-02, -5.6888e-02],
          [-5.0737e-02, -4.4106e-02,  2.3893e-02,  ..., -2.1544e-02,
           -3.9290e-02, -2.4504e-02],
          [-4.4710e-02, -5.5710e-02, -4.2813e-03,  ..., -4.3810e-02,
           -1.9410e-02,  1.8635e-02],
          ...,
          [ 4.4297e-02,  1.7873e-02, -1.6774e-02,  ..., -3.1901e-03,
            8.0985e-03,  1.2061e-02],
          [ 5.7042e-03, -2.7436e-02, -1.7318e-02,  ..., -7.8742e-03,
           -2.8842e-02,  3.1161e-02],
          [ 3.4395e-02,  6.4042e-02, -5.8162e-03,  ...,  6.0890e-02,
            5.3181e-02, -3.0882e-02]],

         [[-1.1184e-02, -1.1377e-02, -2.7410e-02,  ...,  2.4217e-02,
            3.1656e-02, -2.9425e-02],
          [-4.3129e-02,  3.8920e-02, -1.4965e-02,  ...,  8.4295e-03,
           -1.4563e-02,  5.4973e-02],
          [-1.7095e-02,  6.4369e-03,  3.00

In [18]:
model_encoder.update_weights()

updating weights function


In [19]:
model_encoder.show_one_layer_weights(0)

This is the one layer show function
i is:  0
The weights are:  Parameter containing:
tensor([[[[-1.6085e-25, -1.6363e-25, -1.6663e-25,  ..., -1.7430e-25,
           -1.7557e-25, -1.7903e-25],
          [-2.2771e-25, -2.5256e-25, -2.5323e-25,  ..., -2.6019e-25,
           -2.5946e-25, -2.5337e-25],
          [-2.5053e-25, -2.6664e-25, -2.5478e-25,  ..., -2.7490e-25,
           -2.6799e-25, -2.6038e-25],
          ...,
          [-2.6943e-25, -2.8397e-25, -2.8483e-25,  ..., -3.0378e-25,
           -2.9532e-25, -2.8298e-25],
          [-2.8312e-25, -2.8671e-25, -2.8712e-25,  ..., -2.9985e-25,
           -2.9239e-25, -2.9911e-25],
          [-2.9684e-25, -2.9257e-25, -2.9676e-25,  ..., -3.0312e-25,
           -3.0702e-25, -3.0488e-25]],

         [[-1.6082e-25, -1.6399e-25, -1.6598e-25,  ..., -1.7378e-25,
           -1.7550e-25, -1.7926e-25],
          [-2.2830e-25, -2.5180e-25, -2.5322e-25,  ..., -2.6058e-25,
           -2.5928e-25, -2.5301e-25],
          [-2.5050e-25, -2.6661e-25, -2.55

In [20]:
########################################################
# Calculate KDE using sklearn
from sklearn.neighbors import KernelDensity

#Get encoded output of input images = Latent space
# encoded_images = model_encoder(images)
encoded_images = []

for i in range(len(dataset)):
    X = dataset[i]
    image_in_tensor = X[0]
    image_in_tensor = image_in_tensor.cuda()            # Because GPU is being used
    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    encoded_images.append(Y)


c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [21]:
print(len(encoded_images))
print(type(encoded_images))
print(type(encoded_images[0]))

179
<class 'list'>
<class 'torch.Tensor'>


See that it is 179 length because it corresponds to all the images that belong to the training dataset. 
 

Convert "encoded_images" to a np array

In [22]:
np_encoded_images = []
for i in range (len(encoded_images)):
    # np_conversion = encoded_images[i].detach().numpy()        # If not using GPU
    np_conversion = encoded_images[i].cpu().detach().numpy()    # If using GPU
    np_encoded_images.append(np_conversion)
np_encoded_images = np.array(np_encoded_images)
print(type(np_encoded_images))

<class 'numpy.ndarray'>


In [23]:
print(np_encoded_images.shape)

(179, 450, 1, 1)


Also, see above the shape of the representation of the original images has been lowered to (1, 1) as specified in the model structure. The number 450, on the other hand, corresponds to the channels of the image; this value started at 3 and layer by layer it incremented until reaching 450.

Now, we have to flatten the data in order to apply kernel density on it.

In [24]:
model_encoder_output_shape = (450,1,1)
print(model_encoder_output_shape)

(450, 1, 1)


In [25]:
out_vector_shape = model_encoder_output_shape[0]*model_encoder_output_shape[1]*model_encoder_output_shape[2]
encoded_images_vector = [np.reshape(img, (out_vector_shape)) for img in np_encoded_images]


In [26]:
len(encoded_images_vector)

179

In [27]:
encoded_images_vector[0].shape

(450,)

In [28]:
kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(encoded_images_vector)

The above function fits a kernel density estimation to the data that is provided, that is, to the "encoded_images_vector" variable. It does so using a Guassian kernel of bandwidth 0.2.

The badnwidth parameter affects on how the selected kernel will fit each sample of the given data. For example for the case in which the kernel is a Gaussian distribution, the bandwidth parameter would affect in how thin or wide is the Gaussian distribution.

At this point in the variable "kde" we have some numbers that are the result of fitting Gaussian functions to the given data points in the variable "encoded_images_vecotr". We will use the "kde" variable later for scoring with it, some given data points; the scoring will be given depending on how similar are the given data points to the ones that it had estimated.

Here below, it is shown the kde values corresponding to each encoded sample:

In [29]:
density_vals = kde.score_samples(encoded_images_vector)
print(density_vals)

[305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 305.53733485 305.53733485 305.53733485 305.53733485 305.53733485
 306.23048203 306.23048203 306.23048203 306.23048203 306.23048203
 306.23048203 306.23048203 306.2304823  306.2304823  306.23048203
 306.23048203 306.23048203 306.23048203 306.23048203 306.23048203
 306.23048

Notice that the above density values are pretty much the same among them. This has to do with the new model that was trained. The previous model from the "Pt_latentSpace_DS1" program, makes these value to be more different among them.

From here, the mean and standard deviation of these values are computed

In [30]:
average_density = np.mean(density_vals)
stdev_density = np.std(density_vals)
print("The avg of the density values is: ", average_density)
print("The stdev_density of the density values is: ", stdev_density)

The avg of the density values is:  305.6925089493738
The stdev_density of the density values is:  0.28860324374992247


As a side note, the density calculation, was computed, in the video reference (https://www.youtube.com/watch?v=q_tpFGHiRgg&t=1140s&ab_channel=DigitalSreeni), in this manner.  

In [31]:
#Calculate density and reconstruction error to find their means values for
density_lst = []

X = dataset[0]
image_in_tensor = X[0]

model_encoder_output_shape = (450,1,1)          # This is the shape of the last layer of the encoder model
out_vector_shape = model_encoder_output_shape[0]*model_encoder_output_shape[1]*model_encoder_output_shape[2]

for i in range(len(dataset)):
    X = dataset[i]
    image_in_tensor = X[0]
    image_in_tensor = image_in_tensor.cuda()            # Because GPU is being used
    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    # np_converted_encoded_img = Y.detach().numpy()     # If GPU is not used
    np_converted_encoded_img = Y.cpu().detach().numpy() # If GPU is used
    flattened = np.reshape(np_converted_encoded_img, (out_vector_shape))
    density = kde.score_samples([flattened])[0]
    density_lst.append(density)
average_density = np.mean(np.array(density_lst))  
stdev_density = np.std(np.array(density_lst)) 
print("The avg of the density values is: ", average_density)
print("The stdev_density of the density values is: ", stdev_density)


c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to b

The avg of the density values is:  305.6925089493738
The stdev_density of the density values is:  0.28860324374992247


c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to b

This block of code above is not relevant to execute, since it only shows an alternative way of computing the density mean and std values. See that the results from this code block and the one above of it, are the same. 

Now, it will be shown the density mean and std deviation of the set of anomalies samples

In [32]:
data_anomalies = '../../../Images/BottleStoodUp_atNight/Anomalies2.0'      #This is for the home laptop
# data_anomalies = '../../../Images/BottleStoodUp_atNight/Anomalies2.0'      #This is for the work laptop
transform_characteristics = transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize(255),
                                                transforms.CenterCrop(224)])
dataset_anomalies = datasets.ImageFolder(data_anomalies, transform=transform_characteristics)
dataloader_anomalies = torch.utils.data.DataLoader(dataset_anomalies, batch_size=32, shuffle=True)

In [33]:
#Get encoded output of input images = Latent space
encoded_anomalies_images = []

for i in range(len(dataset_anomalies)):
    X = dataset_anomalies[i]
    image_in_tensor = X[0]
    image_in_tensor = image_in_tensor.cuda()            # Because GPU is being used
    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    encoded_anomalies_images.append(Y)


c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [34]:
np_encoded_anomaly_images = []
for i in range (len(encoded_anomalies_images)):
    # np_conversion = encoded_anomalies_images[i].detach().numpy()      # If GPU is not used
    np_conversion = encoded_anomalies_images[i].cpu().detach().numpy()    # If GPU is used
    np_encoded_anomaly_images.append(np_conversion)
np_encoded_anomaly_images = np.array(np_encoded_anomaly_images)
print(type(np_encoded_anomaly_images))

<class 'numpy.ndarray'>


In [35]:
encoded_anomaly_images_vector = [np.reshape(img_encoded, (out_vector_shape)) for img_encoded in np_encoded_anomaly_images]


In [36]:
density_vals_anomalies = kde.score_samples(encoded_anomaly_images_vector)
print(density_vals_anomalies)

[298.60225774  63.75064794 224.22969716 284.25558446 280.04700797
 238.75390245]


In [37]:
average_density_anomalies = np.mean(density_vals_anomalies)
stdev_density_anomalies = np.std(density_vals_anomalies)
print("The avg of the density values is: ", average_density_anomalies)
print("The stdev_density of the density values is: ", stdev_density_anomalies)

The avg of the density values is:  231.6065162868862
The stdev_density of the density values is:  79.46480364757964


See that the std deviation along with the mean of these density values will overlap the mean of the non-anomaly images. 

Considering the situation of the overlapping commented above, given a density value, it will be built a function that:
- Assigns a percentage value according to the proximity to the mean of the non-anomaly images. For example: 
    - If the densitiy value is 305.69 (the mean of the density values of the non-anomaly images), then this density value should have 100% chance to be considered as non-anomaly.
    - If the densitiy value is 305.4014 (the mean of the density values of the non-anomaly minus the std deviation of the same set), then this density value should have 50% chance to be considered as non-anomaly.
- Assigns a percentage value according to the proximity to the mean of the anomaly images. For example: 
    - If the densitiy value is 231.6065 (the mean of the density values of the anomaly images), then this density value should have 100% chance to be considered as an anomaly image.
    - If the densitiy value is 311.0713 (the mean of the density values of the anomaly plus the std deviation of the same set), then this density value should have 50% chance to be considered as an anomaly.
- The two percentage values from above will be summed up assigning the following weights to the equation:

        = perc_NOanomaly*0.75 + perc_anomaly*0.25
        
    More weight is assigned to the non-anomaly images because there are more samples of this kind of images.
- After the weighted sum, the result will be subtracted from 100, to finally output the probability of an image to be an anomaly image. 

In [60]:
def ranges_mapper(value, leftMin, leftMax, rightMin, rightMax):
    # if(value>leftMax):
    #     return rightMax
    # Figure out how 'wide' each range is
    leftSpan = leftMax - leftMin
    rightSpan = rightMax - rightMin

    # Convert the left range into a 0-1 range (float)
    valueScaled = float(value - leftMin) / float(leftSpan)

    # Convert the 0-1 range into a value in the right range.
    return rightMin + (valueScaled * rightSpan)

In [87]:
def map_kde2prob_list(input_list):
    threshold_NOanomaly = average_density          # The mean of the density values corresponding to the non-anomaly images
    std_dev_NOanomaly = stdev_density            # The std deviation of the density values corresponding to the non-anomaly images

    threshold_anomaly = average_density_anomalies          # The mean of the density values corresponding to the anomaly images
    std_dev_anomaly = stdev_density_anomalies            # The std deviation of the density values corresponding to the anomaly images
    prob_score_list = []

    for i in range (len(input_list)):
        score_NOanomaly = input_list[i] - threshold_NOanomaly

        perc_NOanomaly = ranges_mapper(abs(score_NOanomaly), 0, std_dev_NOanomaly, 100, 50)
        if perc_NOanomaly<0:
            perc_NOanomaly = 0
        if perc_NOanomaly>100:
            perc_NOanomaly = 100

        score_anomaly = input_list[i] - threshold_anomaly

        perc_anomaly = ranges_mapper(abs(score_anomaly), 0, std_dev_anomaly, 100, 50)
        if perc_anomaly<0:
            perc_anomaly = 0
        if perc_anomaly>100:
            perc_anomaly = 100
        
        prob_score = 100 - (perc_NOanomaly*0.75+perc_anomaly*0.25)
        prob_score_list.append(prob_score)
    return prob_score_list

In [88]:
def map_kde2prob(value):
    threshold_NOanomaly = average_density          # The mean of the density values corresponding to the non-anomaly images
    std_dev_NOanomaly = stdev_density            # The std deviation of the density values corresponding to the non-anomaly images

    threshold_anomaly = average_density_anomalies          # The mean of the density values corresponding to the anomaly images
    std_dev_anomaly = stdev_density_anomalies            # The std deviation of the density values corresponding to the anomaly images
    score_NOanomaly = value - threshold_NOanomaly

    perc_NOanomaly = ranges_mapper(abs(score_NOanomaly), 0, std_dev_NOanomaly, 100, 50)
    if perc_NOanomaly<0:
        perc_NOanomaly = 0
    if perc_NOanomaly>100:
        perc_NOanomaly = 100

    score_anomaly = value - threshold_anomaly

    perc_anomaly = ranges_mapper(abs(score_anomaly), 0, std_dev_anomaly, 100, 50)
    if perc_anomaly<0:
        perc_anomaly = 0
    if perc_anomaly>100:
        perc_anomaly = 100
    
    prob_score = 100 - (perc_NOanomaly*0.75+perc_anomaly*0.25)
  
    return prob_score

In [89]:
print(map_kde2prob(231.6065+79.4648))
# print(map_kde2prob(231.6065+75.4648))

87.5


In [90]:
def computePred(kde_value):
    pred = 0
    prob_anomaly = map_kde2prob(kde_value)/100
    if prob_anomaly > 0.5:
        pred = 1
    return pred

In [91]:
print("The prob of the kde value of being anomaly image is: ", map_kde2prob(average_density+stdev_density))
print("Given the probability, it is actually predicted as:", computePred(average_density+stdev_density))

The prob of the kde value of being anomaly image is:  50.17501155625721
Given the probability, it is actually predicted as: 1


Testing the kde thresholds 

In [92]:
test_dir = '../../../Images/BottleStoodUp_atNight/Evaluation'      #This is for the home laptop
transform_characteristics = transforms.Compose([transforms.ToTensor(),
                                                transforms.Resize(255),
                                                transforms.CenterCrop(224)])

dataset_test = datasets.ImageFolder(test_dir, transform=transform_characteristics)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=32, shuffle=True)
classes = ('non-anomaly','anomaly')

In [93]:
#Get encoded output of input images = Latent space
encoded_test_imgs = []


for i in range(len(dataset_test)):
    X = dataset_test[i]
    image_in_tensor = X[0]
    image_in_tensor = image_in_tensor.cuda()     
    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    # np_conversion = Y.detach().numpy()
    np_conversion = Y.cpu().detach().numpy()
    encoded_test_imgs.append(np_conversion)
np_encoded_test_images = np.array(encoded_test_imgs)
print(type(np_encoded_test_images))

c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


<class 'numpy.ndarray'>


In [94]:
encoded_test_images_vector = [np.reshape(img, (out_vector_shape)) for img in np_encoded_test_images]
print(len(encoded_test_images_vector))

30


In [95]:
density_vals_test = kde.score_samples(encoded_test_images_vector)
print(density_vals_test)

[  -350.67522441   -693.63549459  -2023.2241612   -2184.82534615
  -2799.39007047  -3267.02837392  -5174.08288287  -2772.38539377
  -1147.55379623  -2420.5401337     116.3121348     146.8796439
    270.02545513    -76.71125116   -564.48883414     74.13091853
 -20701.70905962 -16413.89368952  -1592.81200715   -267.54464273
   -236.72965081   -544.44303225   -661.39524567   -561.63829861
   -609.95590391   -950.1247644    -915.71611702  -1473.93517446
  -1746.24829226  -1102.07361035]


The above shown values are REALLY strange. It was not expected to have negative values in the density numbers!

In [96]:
prob_test = map_kde2prob_list(density_vals_test)

In [97]:
print(prob_test)

[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 93.13607490326693, 88.32773380461066, 81.04339203237086, 100.0, 100.0, 99.77127946392781, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]


In [98]:
y_pred = []
y_true = []

# Grabbing only the first image of the anomalies dataset
X = dataset_test[0]
image_in_tensor = X[0]
 
n_features = len(image_in_tensor[0])  # Get the size of one image of the anomaly images dataset. This is supposed to be 224
for i in range(len(dataset_test)):
    X = dataset_test[i]
    image_in_tensor = X[0]
    image_in_tensor = image_in_tensor.cuda() 
    ground_truth = X[1]

    with torch.no_grad():
        Y = model_encoder(image_in_tensor)  # should be same as X
    np_converted_encoded_img = Y.cpu().detach().numpy()
    flattened = np.reshape(np_converted_encoded_img, (out_vector_shape))
    density = kde.score_samples([flattened])[0]
    prediction = computePred(density)
    y_pred.append(prediction) # Save Prediction
    y_true.append(ground_truth) # Save Truth
print(y_true)
print(y_pred)

c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to b

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
c:\Users\alexj\.conda\envs\tfm_3\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to b

Above, it can be seen that all the images are predicted to be anomaly images. This is good for the TPR but it is terrible for the FPR. 